# 利用 OpenAI Python 函式庫整合 OpenRouter API —— 以 Quasar Alpha 為範例

OpenRouter 提供了與 OpenAI API 相容的統一介面，讓開發者可以快速串接多種大型語言模型（LLM）。本教學將示範如何利用 OpenAI 函式庫整合 OpenRouter API，實現對話生成與單次問答的功能。

---

## 1. 環境設定與客戶端初始化

首先，你需要安裝 OpenAI Python 函式庫，可以透過 pip 指令安裝：

In [24]:
!pip install openai

接著透過下列程式碼初始化客戶端，並指定 API 伺服器 URL 與你的 API 金鑰。


In [3]:
from openai import OpenAI

client = OpenAI(
    base_url='https://openrouter.ai/api/v1/',  # 設定 OpenRouter API 的 URL
    api_key='YOUR_API_KEY_HERE',                # 請填入你從 OpenRouter 取得的 API 金鑰
)

# 2. Chat Completion API（對話生成）

### 2.1 純文字聊天

以下程式碼示範如何向 Quasar Alpha 建立一個純文字聊天請求，更多模型參數設置可以[參考](https://openrouter.ai/docs/api-reference/parameters)：

In [33]:
# 使用 chat API 建立一個聊天請求
chat_completion = client.chat.completions.create(
    messages=[
        {
            'role': 'user',                         # 設定角色為「使用者」
            'content': '請隨機給一個數字',         # 輸入你要詢問的問題
        }
    ],
    model='openrouter/quasar-alpha',              # 指定使用 Quasar Alpha 模型
    temperature=1,  # 設定溫度值，影響回答的隨機性
    max_tokens=512,  # 設定回應的最大 Token 數（字數限制）
    top_p=1  # 設定 Top-p，用於控制回應的多樣性
)

# 取得並印出模型回應的內容
print(chat_completion.choices[0].message.content)

47


### 2.2 文字與影像混合輸入的聊天

以下範例示範如何同時傳送文字與以 Base64 編碼的影像資料：

In [27]:
import base64

# 讀取本機圖片並轉為 Base64 格式
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# 圖片路徑
image_path = "./image.png"
base64_image = encode_image(image_path)

response = client.chat.completions.create(
    model="openrouter/quasar-alpha",           # 使用支援 Vision 功能的 Quasar Alpha 模型（若模型具備相關功能）
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "這張圖片裡有什麼？"},
                {"type": "image_url", "image_url": f"data:image/jpeg;base64,{base64_image}"}
            ]
        }
    ],
    max_tokens=300,
)

# 取得並印出回應內容
print(response.choices[0].message.content)

這張圖片中有一位穿著畢業袍的人，手裡拿著一束花。背景是一片綠意盎然的草地和樹木，看起來像是在戶外的公園或校園。這應該是一張關於畢業典禮的照片。


In [32]:
# 亦或是直接給圖像網址也能理解圖像內容
response = client.chat.completions.create(
    model="openrouter/quasar-alpha",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "這張圖片裡有什麼？"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
                    }
                }
            ]
        }
    ],
    max_tokens=300,
)

# 取得並印出回應內容
print(response.choices[0].message.content)

這張圖片中是一片寬廣的綠色草地，中央有一條木質步道延伸向遠方。天氣晴朗，藍天上有一些白色的雲朵，整體景色十分寧靜自然，展現出郊外或濕地的風光。


## 3. Completion API (單次問答)

如果不需要多輪對話，也可以直接使用單次問答 API。例如，以下程式以提示「美國首都在哪裡？」來取得結果：


In [28]:
completion = client.completions.create(
    model="openrouter/quasar-alpha",  # 指定使用 Quasar Alpha 模型
    prompt="美國首都在哪裡？",          # 提示詞
)

# 取得並印出模型回應的內容
print(completion.choices[0].text)

美國的首都位於華盛頓哥倫比亞特區（Washington, D.C.）。
